In [ ]:
import scienceplots
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use(['science', 'grid'])

In [ ]:
# read data

df_all_model = pd.read_parquet('../scripts/nway_csc21_gaia3_full_neg_study_dis_niter200.parquet')